# Lending Club Case Study
### Problem Statement

When the company receives a loan application, the company has to make a decision for loan approval based on the applicant’s profile. Two types of risks are associated with the bank’s decision:

1. If the applicant is likely to repay the loan, then not approving the loan results in a loss of business to the company

2. If the applicant is not likely to repay the loan, i.e. he/she is likely to default, then approving the loan may lead to a financial loss for the company

In [64]:
# importing all library used for this analysis
import pandas as pd
import numpy as np

# Importing Graphical Ploting library for Data Visualiazation 
import matplotlib.pyplot as plt

from datetime import datetime as dt
import seaborn as sns
sns.set()

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',150)
import warnings
warnings.filterwarnings('ignore')

#### Importing Data File

In [27]:
# low_memory=False helps to read the file in one go and not in chucks, 
# With in the data there exsits columns with mixed data type (dtype) lowering the perofomance.
# reading loan data as pandas dataframe
loan = pd.read_csv("loan.csv",low_memory=False)
loan.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [28]:
loan.shape

(39717, 111)

In [5]:
#Check the datatypes of all the columns of the dataframe
loan.info()
loan.columns.values


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(74), int64(13), object(24)
memory usage: 33.6+ MB


array(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       'emp_title', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan',
       'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d',
       'last_credit_pull_d', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'application_type',
       'annual_inc_joint', 'dti_joint', 'verification_status_joint

#### Missing Value Treatment

In [31]:
# Finding the percentage of the Null values and Sorting them (Percentage give a better understanding the count of null values)
missing_value_per = loan.isnull().mean()*100
result_desc = missing_value_per.sort_values(ascending = False)

# Computing the count of columns where null value of the coulmn is equal to 100% 
(result_desc != 100).sum()

57

In [33]:
#Check the number of null values in the columns
loan.isnull().sum()/len(loan)

id                            0.000000
member_id                     0.000000
loan_amnt                     0.000000
funded_amnt                   0.000000
funded_amnt_inv               0.000000
                                ...   
tax_liens                     0.000982
tot_hi_cred_lim               1.000000
total_bal_ex_mort             1.000000
total_bc_limit                1.000000
total_il_high_credit_limit    1.000000
Length: 111, dtype: float64

Since we have too many columns, lets find the percentage of missing data in each column and print columns which has more that 40 percent missing data

In [37]:
# Threshold Setting
nullcol_thres = 0.4*len(loan)
null_col_head = [col for col in loan.columns if loan[col].isnull().sum()>=nullcol_thres]

# Dropping Columns
filtered_df = loan.drop(null_col_head, axis=1)

In [40]:
# Dropping Columns
filtered_df.head(10)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,171.62,16-May,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,119.66,13-Sep,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,649.91,16-May,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,357.48,16-Apr,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,67.79,16-May,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
5,1075269,1311441,5000,5000,5000.0,36 months,7.90%,156.46,A,A4,...,161.03,16-Jan,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
6,1069639,1304742,7000,7000,7000.0,60 months,15.96%,170.08,C,C5,...,1313.76,16-May,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
7,1072053,1288686,3000,3000,3000.0,36 months,18.64%,109.43,E,E1,...,111.34,14-Dec,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
8,1071795,1306957,5600,5600,5600.0,60 months,21.28%,152.39,F,F2,...,152.39,12-Aug,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
9,1071570,1306721,5375,5375,5350.0,60 months,12.69%,121.45,B,B5,...,121.45,13-Mar,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0


___Analysing the dataframe after dropping the columns___

In [42]:
missing_value_per = filtered_df.isnull().mean()*100
result_desc = missing_value_per.sort_values(ascending = False)

In [45]:
result_desc
filtered_df['int_rate'].value_counts()

10.99%    956
13.49%    826
11.49%    825
7.51%     787
7.88%     725
         ... 
18.36%      1
16.96%      1
16.15%      1
16.01%      1
17.44%      1
Name: int_rate, Length: 371, dtype: int64

In [48]:
filtered_df['int_rate'] = filtered_df['int_rate'].replace('%','').astype(float).round(0)
filtered_df['revol_util'] = filtered_df['revol_util'].replace('%','').astype(float).round(0)

In [50]:
filtered_df['int_rate']

0        11.0
1        15.0
2        16.0
3        13.0
4        13.0
         ... 
39712     8.0
39713    10.0
39714     8.0
39715     7.0
39716    14.0
Name: int_rate, Length: 39717, dtype: float64

In [52]:
filtered_df['term'].value_counts()

 36 months    29096
 60 months    10621
Name: term, dtype: int64

In [54]:
date_coulms = filtered_df[['issue_d','earliest_cr_line','last_pymnt_d','last_credit_pull_d']]
for col in date_coulms:
    filtered_df[col] = pd.to_datetime(filtered_df[col], format = '%b-%y',errors='coerce')

In [59]:
filtered_df[['earliest_cr_line','issue_d','last_pymnt_d','last_credit_pull_d']].head()

,earliest_cr_line,issue_d,last_pymnt_d,last_credit_pull_d
0,1985-01-01,NaT,NaT,NaT
1,1999-04-01,NaT,NaT,NaT
2,NaT,NaT,NaT,NaT
3,1996-02-01,NaT,NaT,NaT
4,1996-01-01,NaT,NaT,NaT


In [60]:
# Droping Desc as 30% of the values are null and within the url the data present with in the coulmn have no analysis power 
filtered_df.drop(columns=['desc','url'], inplace = True)

In [62]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   id                          39717 non-null  int64         
 1   member_id                   39717 non-null  int64         
 2   loan_amnt                   39717 non-null  int64         
 3   funded_amnt                 39717 non-null  int64         
 4   funded_amnt_inv             39717 non-null  float64       
 5   term                        39717 non-null  object        
 6   int_rate                    39717 non-null  float64       
 7   installment                 39717 non-null  float64       
 8   grade                       39717 non-null  object        
 9   sub_grade                   39717 non-null  object        
 10  emp_title                   37258 non-null  object        
 11  emp_length                  38642 non-null  object    

In [65]:
filtered_df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599,5000,5000,4975.0,36 months,11.0,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,NaT,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0,1985-01-01,1,3,0,13648,84.0,9,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,NaT,171.62,NaT,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
1,1077430,1314167,2500,2500,2500.0,60 months,15.0,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,NaT,Charged Off,n,car,bike,309xx,GA,1.00,0,1999-04-01,5,3,0,1687,9.0,4,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,NaT,119.66,NaT,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
2,1077175,1313524,2400,2400,2400.0,36 months,16.0,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,NaT,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0,NaT,2,2,0,2956,98.0,10,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,NaT,649.91,NaT,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
3,1076863,1277178,10000,10000,10000.0,36 months,13.0,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,NaT,Fully Paid,n,other,personel,917xx,CA,20.00,0,1996-02-01,1,10,0,5598,21.0,37,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,NaT,357.48,NaT,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
4,1075358,1311748,3000,3000,3000.0,60 months,13.0,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,NaT,Current,n,other,Personal,972xx,OR,17.94,0,1996-01-01,0,15,0,27783,54.0,38,f,524.06,524.06,3513.330000,3513.33,2475.94,1037.39,0.00,0.00,0.00,NaT,67.79,NaT,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0


In [73]:
filtered_df.describe()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,pub_rec_bankruptcies,tax_liens
count,3.971700e+04,3.971700e+04,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,3.971700e+04,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39667.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39020.000000,39678.0
mean,6.831319e+05,8.504636e+05,11219.443815,10947.713196,10397.448877,12.011229,324.561922,6.896893e+04,13.315130,0.146512,0.869200,9.294408,0.055065,13382.528086,48.833514,22.088828,51.227887,50.989768,12153.596544,11567.149118,9793.348813,2263.663172,1.363015,95.221624,12.406112,2678.826162,0.043260,0.0
std,2.106941e+05,2.656783e+05,7456.670694,7187.238670,7128.450440,3.717187,208.874874,6.379377e+04,6.678594,0.491812,1.070219,4.400282,0.237200,15885.016641,28.339127,11.401709,375.172839,373.824457,9042.040766,8942.672613,7065.522127,2608.111964,7.289979,688.744771,148.671593,4447.136012,0.204324,0.0
min,5.473400e+04,7.069900e+04,500.000000,500.000000,0.000000,5.000000,15.690000,4.000000e+03,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,5.162210e+05,6.667800e+05,5500.000000,5400.000000,5000.000000,9.000000,167.020000,4.040400e+04,8.170000,0.000000,0.000000,6.000000,0.000000,3703.000000,25.000000,13.000000,0.000000,0.000000,5576.930000,5112.310000,4600.000000,662.180000,0.000000,0.000000,0.000000,218.680000,0.000000,0.0
50%,6.656650e+05,8.508120e+05,10000.000000,9600.000000,8975.000000,12.000000,280.220000,5.900000e+04,13.400000,0.000000,1.000000,9.000000,0.000000,8850.000000,49.000000,20.000000,0.000000,0.000000,9899.640319,9287.150000,8000.000000,1348.910000,0.000000,0.000000,0.000000,546.140000,0.000000,0.0
75%,8.377550e+05,1.047339e+06,15000.000000,15000.000000,14400.000000,15.000000,430.780000,8.230000e+04,18.600000,0.000000,1.000000,12.000000,0.000000,17058.000000,72.000000,29.000000,0.000000,0.000000,16534.433040,15798.810000,13653.260000,2833.400000,0.000000,0.000000,0.000000,3293.160000,0.000000,0.0
max,1.077501e+06,1.314167e+06,35000.000000,35000.000000,35000.000000,25.000000,1305.190000,6.000000e+06,29.990000,11.000000,8.000000,44.000000,4.000000,149588.000000,100.000000,90.000000,6311.470000,6307.370000,58563.679930,58563.680000,35000.020000,23563.680000,180.200000,29623.350000,7002.190000,36115.200000,2.000000,0.0


In [74]:
filtered_df[['collections_12_mths_ex_med','policy_code','acc_now_delinq','chargeoff_within_12_mths','delinq_amnt']].describe()

KeyError: "None of [Index(['collections_12_mths_ex_med', 'policy_code', 'acc_now_delinq',\n       'chargeoff_within_12_mths', 'delinq_amnt'],\n      dtype='object')] are in the [columns]"

In [71]:
# Dropping these coloumns where from the above describe function 
# There is no comparable or catagorical data present whaich can be vaulabe for the analysis 
filtered_df.drop(columns=['collections_12_mths_ex_med','policy_code','acc_now_delinq','chargeoff_within_12_mths','delinq_amnt'], inplace = True)

KeyError: "['collections_12_mths_ex_med', 'policy_code', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt'] not found in axis"

In [ ]:
filtered_df.set_index('id',inplace = True)

In [75]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 47 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   id                       39717 non-null  int64         
 1   member_id                39717 non-null  int64         
 2   loan_amnt                39717 non-null  int64         
 3   funded_amnt              39717 non-null  int64         
 4   funded_amnt_inv          39717 non-null  float64       
 5   term                     39717 non-null  object        
 6   int_rate                 39717 non-null  float64       
 7   installment              39717 non-null  float64       
 8   grade                    39717 non-null  object        
 9   sub_grade                39717 non-null  object        
 10  emp_title                37258 non-null  object        
 11  emp_length               38642 non-null  object        
 12  home_ownership           39717 n

In [76]:
numerical_columns = filtered_df.select_dtypes(include = ['float64','int64']).columns
numerical_columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'int_rate', 'installment', 'annual_inc', 'dti', 'delinq_2yrs',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'pub_rec_bankruptcies', 'tax_liens'],
      dtype='object')

In [78]:
string_coulmuns = filtered_df.select_dtypes(include = ['object']).columns
string_coulmuns
filtered_df[string_coulmuns]

,term,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,initial_list_status,application_type
0,36 months,B,B2,NaN,10+ years,RENT,Verified,Fully Paid,n,credit_card,Computer,860xx,AZ,f,INDIVIDUAL
1,60 months,C,C4,Ryder,< 1 year,RENT,Source Verified,Charged Off,n,car,bike,309xx,GA,f,INDIVIDUAL
2,36 months,C,C5,NaN,10+ years,RENT,Not Verified,Fully Paid,n,small_business,real estate business,606xx,IL,f,INDIVIDUAL
3,36 months,C,C1,AIR RESOURCES BOARD,10+ years,RENT,Source Verified,Fully Paid,n,other,personel,917xx,CA,f,INDIVIDUAL
4,60 months,B,B5,University Medical Group,1 year,RENT,Source Verified,Current,n,other,Personal,972xx,OR,f,INDIVIDUAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39712,36 months,A,A4,FiSite Research,4 years,MORTGAGE,Not Verified,Fully Paid,n,home_improvement,Home Improvement,802xx,CO,f,INDIVIDUAL
39713,36 months,C,C1,"Squarewave Solutions, Ltd.",3 years,RENT,Not Verified,Fully Paid,n,credit_card,Retiring credit card debt,274xx,NC,f,INDIVIDUAL
39714,36 months,A,A4,NaN,< 1 year,MORTGAGE,Not Verified,Fully Paid,n,debt_consolidation,MBA Loan Consolidation,017xx,MA,f,INDIVIDUAL
39715,36 months,A,A2,NaN,< 1 year,MORTGAGE,Not Verified,Fully Paid,n,other,JAL Loan,208xx,MD,f,INDIVIDUAL


In [79]:
# Converting data cloumns into categorical data
filtered_df[['term', 'grade', 'sub_grade', 'emp_length','home_ownership', 'verification_status', 'loan_status', 'pymnt_plan','application_type']] = filtered_df[['term', 'grade', 'sub_grade', 'emp_length','home_ownership', 'verification_status', 'loan_status', 'pymnt_plan','application_type']].astype('category')

In [80]:
filtered_df['grade'].astype('category')

0        B
1        C
2        C
3        C
4        B
        ..
39712    A
39713    C
39714    A
39715    A
39716    E
Name: grade, Length: 39717, dtype: category
Categories (7, object): ['A', 'B', 'C', 'D', 'E', 'F', 'G']

In [81]:
string_coulmuns = filtered_df.select_dtypes(include = ['object']).columns
string_coulmuns

Index(['emp_title', 'purpose', 'title', 'zip_code', 'addr_state',
       'initial_list_status'],
      dtype='object')

In [82]:
filtered_df[string_coulmuns].value_counts()

emp_title                            purpose             title                                 zip_code  addr_state  initial_list_status
Associated Mortgage Group, Inc.      debt_consolidation  Consolidation                         972xx     OR          f                      2
Wells Fargo                          small_business      Small Business                        782xx     TX          f                      2
SOUTHEASTERN INTEGRATED MEDICAL, PL  debt_consolidation  CONSOLIDATION LOAN                    326xx     FL          f                      2
Burrell School District              debt_consolidation  consolidate                           150xx     PA          f                      2
Sheridan Auto Group                  credit_card         Credit Card Loan                      197xx     DE          f                      2
                                                                                                                                           ..
Government 

In [ ]:
# Data cleaning completed

#### Sanity Checks